# Defining the Fitness and Criticality function

## Fitness Function


In this tutorial we show how to specify two fitness functions for the AEB case study: 

- F1) This function evaluates the distance between the ego vehicle and the pedestrian.
- F2) This function outputs the velocity of the ego vehicle at the time of the minimal distance to the pedestrian.

By minimizing F1) and maximizing F2) we want to identify failure-revealing test cases. 

OpenSBT provdes the interface `Fitness` to implement fitness functions. `Fitness` provides the `eval` function which needs to be implemented with concrete evaluation instructions. The `eval`function receives as input an instance of  [`SimulationOutput`](https://git.fortiss.org/opensbt/opensbt-core/-/blob/main/simulation/simulator.py#L40-62) and returns a scalar or a vector valued output. 

A scalar value is returned when a single objective is to be optimized, otherwise a vector valued output is returned, such as in our example. Further `Fitness` contains the `min_max` function which has to return a tuple of `min` or `max` strings indicating whether the corresponding objective function is minimized or maximized. 
In the `name` function we provide names for both fitness functions to be shown in output artefacts.

The implementation of the examplary fitness function in OpenSBT then looks as follows:



In [ ]:
from typing import Tuple
from evaluation.fitness import Fitness
from simulation.simulator import SimulationOutput
from utils import geometric
import numpy as np

class FitnessMinDistanceVelocity(Fitness):
    @property
    def min_or_max(self):
        return "min", "max"

    @property
    def name(self):
        return "Min distance", "Velocity at min distance"

    def eval(self, simout: SimulationOutput) -> Tuple[float]:
        traceEgo = simout.location["ego"]                     # The actor names have to match the names written into the simulationoutput instance by the simulation adapter
        tracePed = simout.location["adversary"]              

        ind_min_dist = np.argmin(geometric.distPair(traceEgo, tracePed))

        # distance between ego and other object
        distance = np.min(geometric.distPair(traceEgo, tracePed))

        # speed of ego at time of the minimal distance
        speed = simout.speed["ego"][ind_min_dist]

        return (distance, speed)


## Criticality Function

Beside a fitness function,often a safety requirement/testing oracle is given specifying which test inputs are failure-revealing or not. In our example, such a safety requirement is: 

_<p style="text-align: center;">A test case is critical, when F1 is below .5m and ego vehicles velocity is not 0m/s._
    </p>
In OpenSBT, a safety requirements is represented by the interface class [Critical](evaluation/critical.py). `Critical` offers the method `eval`, where specific constraints can be implemented. It received as input the fitness vector in form of an array, and the simulation output for the corresponding test input.

In [ ]:
from evaluation.critical import Critical
from simulation.simulator import SimulationOutput

class CriticalExample(Critical):
    def eval(self, vector_fitness, simout: SimulationOutput = None):
        # Corresponds F1 < 0.5 and F2 > 0 (inverted because minimization is performed)
        if (vector_fitness[0] < 0.5) and (vector_fitness[1] < 0):
            return True
        else:
            return False